In [1]:
%config Completer.use_jedi = False

%load_ext autoreload
%autoreload 2

In [2]:
import os
import sys

sys.path.insert(1, os.path.join(sys.path[0], ".."))

In [3]:
SEED = 123
bar_format = '{desc}{percentage:5.1f}%|{bar:10}{r_bar}'

FILENAME = "{dataset_name}-{planes_segmentation__algo}-mom_{metrics__mom__input}"

In [4]:
import os
from easydict import EasyDict
import pickle
from functools import partial

from tqdm import tqdm

from registration_metrics import (
    seed_everything,
    CARLA_dataset,
    SemanticKITTI_dataset,
    prepare_data,
    sampling_pipeline,
    load_config,
    get_filename_fmt,
    new_value_if_none,
)

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [4]:
def run_experiment(
    seed, config, filename, override=False,
    map_size=10, N_samples=5, cov_scaler=10,
    # start_internal_id=0, end_internal_id=2422, step_internal_id=40,
    start_internal_id=None, end_internal_id=None, step_internal_id=None,
    prepare_func=prepare_data, dataset="CARLA",
    verbose=False
):
    # Load config
    if isinstance(config, str):
        config = load_config(config)

    # Prepare dataset_builder
    if isinstance(dataset, str):
        dataset = dataset.lower()
        assert dataset in ("carla", "semantickitti")
        if dataset == "carla":
            dataset_builder = partial(CARLA_dataset, dataset_coords_type="world")
        elif dataset == "semantickitti":
            dataset_builder = SemanticKITTI_dataset
    else:
        dataset_builder = dataset

    # Set begin, end and step for external ids for dataset
    if isinstance(dataset_builder, CARLA_dataset):
        start_internal_id = new_value_if_none(start_internal_id, 0)
        end_internal_id = new_value_if_none(end_internal_id, 2422)
        step_internal_id = new_value_if_none(step_internal_id, 40)
    elif isinstance(dataset_builder, SemanticKITTI_dataset):
        start_internal_id = new_value_if_none(start_internal_id, 0)
        end_internal_id = new_value_if_none(end_internal_id, 3630)
        step_internal_id = new_value_if_none(step_internal_id, 120)
    else:
        raise ValueError('Unknown dataset_builder type.')

    # Determine dataset name
    dataset_name = type(dataset_builder).__name__
    dataset_name_suffix = "_dataset"
    if dataset_name.endswith(dataset_name_suffix):
        dataset_name = dataset_name[:-len(dataset_name_suffix)]

    # Determine filename for results
    filename_fmt = get_filename_fmt(config)
    filename_fmt["dataset_name"] = dataset_name
    filename = filename.format(**filename_fmt) + ".pkl"
    print(f"Results WILL be saved to \"{filename}\".")

    # Check existance of file for results
    if os.path.exists(filename):
        print(f"File \"{filename}\" for results exists! ", end="")
        if override:
            print("Continuing.")
        else:
            print("Exiting.")
            return

    seed_everything(seed)

    results = {}

    try:
        # Main loop
        for internal_id in tqdm(
            range(start_internal_id, end_internal_id, step_internal_id),
            desc="Internal IDs",
            bar_format=bar_format
        ):
            start_id, end_id = internal_id, internal_id + map_size

            data = dataset_builder(start_id=start_id, end_id=end_id)

            pcs = data.clds
            T_gt = data.poses

            pcs, T_gt, orth_planes, orth_planes_cld_ind, details = prepare_func(
                pcs, T_gt, config)
            details["orth_planes"] = orth_planes
            details["orth_planes_cld_ind"] = orth_planes_cld_ind

            tips = EasyDict(orth_list=orth_planes, config=config)

            save_dict = sampling_pipeline(
                pcs=pcs, T_gt=T_gt, map_tips=tips,
                N_samples=N_samples, cov_scaler=cov_scaler,
                orth_planes_cld_ind=orth_planes_cld_ind,
                verbose=verbose
            )

            results[internal_id, map_size] = {**details, **save_dict}
    except KeyboardInterrupt:
        print("Process was interrupted! Probably something is missed!")

    with open(filename, 'wb') as sfile:
        pickle.dump(results, sfile)
    print(f"Results were saved to \"{filename}\".")


run_CARLA_seq5 = partial(run_experiment, dataset="carla")

# SVD planes extraction, MoM on planes

In [9]:
filename = FILENAME
run_CARLA_seq5(SEED, "svd_mom_planes", filename)

{'metrics': {'mean_map_metric': {'min_nn': 6},
             'mom': {'input': 'planes', 'min_nn': 6}},
 'normals_estimation': {'max_nn': 30, 'max_radius': 1.0},
 'orth_planes_extraction': {'affinity': 'cosine',
                            'dist_threshold': 0.1,
                            'downsample': {'upsample': False,
                                           'use': False,
                                           'voxel_size': 0.6},
                            'from_pc': 'first',
                            'min_clust_size': 10},
 'planes_segmentation': {'algo': 'svd_filter'}}
Results WILL be saved to "CARLA-svd_filter-mom_planes.pkl".
Random seed set as 123


Internal IDs100.0%|██████████| 61/61 [56:29<00:00, 55.56s/it]


Results were saved to "CARLA-svd_filter-mom_planes.pkl".


Посчиталось за 56 минуты 29 секунду.